In [2]:
import pandas as pd
import os
import numpy as np

In [ ]:
mhy7_path = 'data/mhy7/mhy7.tsv'
myo5b_path = 'data/myo5b/myo5b_variants_patho_benign_cadd1.3fullannot_v1.xlsx'
mhy7 = pd.read_csv(mhy7_path,sep='\t')
